#### Given some urls extract items of menu with their prices

Import libraries and OpenAI key

In [1]:
# Scraping
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md

#Handle Greek letters in url
from urllib.parse import urlparse
from urllib.parse import unquote

import openai
#Set API key for OpenAI
openai.api_key = openai_api_key#="sk-....." 

Urls of websites to parse data from - Taken from https://www.geeksforgeeks.org/extract-all-the-urls-from-the-webpage-using-python/

In [2]:
#Urls of websites to parse data from
url = 'https://www.numberone.gr/'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
urls = []
for link in soup.find_all('a'): # Find All <a> Tags
    
    if 'https:' in link.get('href'): #Avoid irrelevant information
        # print(link.get('href'))
        urls.append(link.get('href'))

In [3]:
# [url for url in urls if 'burger' in url]  #Page below missing!!
#https://numberone.gr/?product=%cf%86%cf%84%ce%b9%ce%ac%ce%be%ce%b5-%cf%84%ce%bf-%ce%b4%ce%b9%ce%ba%cf%8c-%cf%83%ce%bf%cf%85-burger

In [4]:
# urls=['https://numberone.gr/?product_cat=pasta','https://numberone.gr/?product_cat=%ce%ba%ce%bb%ce%b1%cf%83%cf%83%ce%b9%ce%ba%ce%b7']
# urls=["https://numberone.gr/?product_cat=streetfood","https://numberone.gr/?product_cat=tisoras",'https://numberone.gr/?product_cat=pasta',
#       'https://numberone.gr/?product_cat=sandwich','https://numberone.gr/?product_cat=%ce%ba%ce%bb%ce%b1%cf%83%cf%83%ce%b9%ce%ba%ce%b7']

Function to parse data from web - Taken from https://github.com/gkamradt/langchain-tutorials/blob/main/data_generation/Using%20LLMs%20To%20Summarize%20Personal%20Research.ipynb

In [5]:
def pull_from_website(url):
    
    # Doing a try in case it doesn't work
    try:
        response = requests.get(url)
    except:
        # In case it doesn't work
        print ("Whoops, error")
        return
    
    # Put your response in a beautiful soup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Get your text
    text = soup.get_text()

    # Convert your html to markdown. This reduces tokens and noise (eg. removes spaces)
    text = md(text)
     
    return text

Parse info from each url, send it to OpenAI, and extract only item with its price

In [6]:
# Store website data in a simple string. Can still be optimized
website_data = ""
final_data=""

for url in urls:
    print("URL: ", url)
    text = pull_from_website(url)

    try:
        text=text.split('Κορυφαία Πιάτα')[0] #Ignore text after this phrase since these are just suggestions
        text=text.split('results') #If this phrase exists, then items to order will also exist.
        if len(text)>1: #If there is a split and not the same text returned,
            text=text[1]
        else:
            continue #If there is no split, then there is no menu

        #Below needed if GPT not used:
        text=text.split('Προσθήκη') #This is used to split individuals items of menu
        text=text[:-1] #Ignore last element with strange characters

    except:
        print("'Κορυφαία πιάτα' not found")

    # print("Original text \n",text) #Original parsed text

    try: #Might result in error if '=' does not exist in url - In that case not valid link with items

        text="\n".join(text)
        print("Non-formatted data: \n",text)

        #Taken from https://www.quora.com/How-do-I-escape-a-unicode-URL-in-Python
        url_name=url.split('=')[1]
        url_name = unquote(url_name)
    
        with open(url_name+".txt", "w", encoding="utf-8") as f: #Save as txt based on section title
            f.write(text)
    
        website_data += text
        website_data += "\n"

        #Send above parsed website to OpenAI - Taken from https://platform.openai.com/docs/guides/gpt/chat-completions-api
        result=openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature = 0.01,
            messages=[
                    {"role": "system", "content": "The following is the menu of a Greek restaurant in a website. \
                        Extract each item of the menu, along with its price. Write in parenthesis any extra information of each item (if provided). \
                        For each item, create a new line in the output. The format of it should be 'item (details) - price'. Take time to find all items.  \
                        The output should be in Greek, as is the original menu. Use lowercase when possible, for both 'item' and 'details'. \
                        Do not change the order of any items of the menu. Do not make things up. \
                        If there are both English and Greek descriptions of an item, keep only the Greek one. \
                        The menu is as follows:"},
                    {"role": "user", "content": text}  
                ]) 
        
        final_data+=result['choices'][0]['message']['content']
        final_data+="\n"

        #Save GPT output to txt
        with open("final_"+url_name+".txt", "w", encoding="utf-8") as f:
            f.write(result['choices'][0]['message']['content'])
            # f.write(final_data)
        
        print("GPT formatted text \n",result['choices'][0]['message']['content']) #Formatted text from GPT
        print('\n')

    except: #When link errors
        print("No menu")
        print('\n')

URL:  https://numberone.gr/
URL:  https://numberone.gr/?page_id=8
URL:  https://numberone.gr/?page_id=8
URL:  https://numberone.gr/?product_cat=streetfood
URL:  https://numberone.gr/?product_cat=burgers
Non-formatted data: 
  Halloumi BurgerΜΠΙΦΤΕΚΙ ΜΟΣΧΑΡΙΣΙΟ , ΧΑΛΟΥΜΙ , ΡΟΚΑ , ΝΤΟΜΑΤΑ , ΜΑΓΙΟΝΕΖΑ €5.30 
 Number 1 BurgerΜΠΙΦΤΕΚΙ ΜΟΣΧΑΡΙΣΙΟ , ΔΙΠΛΟ CHEDDAR , ΑΥΓΟ ΤΗΓΑΝΙΤΟ , ΜΑΡΟΥΛΙ , ΝΤΟΜΑΤΑ , ΚΑΡΑΜΕΛΩΜΕΝΑ ΚΡΕΜΜΥΔΙΑ €5.50 
 Shrimp BurgerΠΑΝΑΡΙΣΜΕΝΗ ΓΑΡΙΔΑ , ΝΤΟΜΑΤΑ , ΚΡΕΜΜΥΔΙ , MΑΡΟΥΛΙ ,  ΑΒΟΚΑΝΤΟ , ΣΩΣ COCKTAIL €5.20 
 Φτιάξε το δικό σου burger! €0.80 
 HamburgerΜΠΙΦΤΕΚΙ, ΝΤΟΜΑΤΑ, ΚΕΤΣΑΠ, ΑΓΓΟΥΡΟΜΑΓΙΟΝΕΖΑ €3.60 
 Cheese burgerΜΠΙΦΤΕΚΙ, ΤΥΡΙ, ΝΤΟΜΑΤΑ, ΚΕΤΣΑΠ, ΑΓΓΟΥΡΟΜΑΓΙΟΝΕΖΑ €3.90 
 Super burgerΜΠΙΦΤΕΚΙ, ΝΤΟΜΑΤΑ, ΜΠΕΪΚΟΝ, ΠΑΡΜΕΖΑΝΑ, ΚΕΤΣΑΠ, ΑΓΓΟΥΡΟΜΑΓΙΟΝΕΖΑ €4.00 
 Mega burgerΔΙΠΛΟ ΜΠΙΦΤΕΚΙ, ΔΙΠΛΟ ΤΥΡΙ, ΜΠΕΪΚΟΝ, ΜΑΡΟΥΛΙ, ΚΡΕΜΜΥΔΙ, ΝΤΟΜΑΤΑ, ΚΕΤΣΑΠ, ΜΑΓΙΟΝΕΖΑ. ΣYNOΔEYETAI ME ΠΑΤΑΤΕΣ, ΣΩΣ €7.90 
 Chicken burgerΜΠΙΦΤΕΚΙ ΚΟΤΟΠΟΥΛΟ, ΤΥΡΙ, ΝΤΟΜΑΤΑ, ΜΑΡΟΥΛΙ, ΣΩΣ €4.00 
 Golden chickenΣΝΙΤΣΕΛ 

In [7]:
with open("final_website_preprocessed.txt", "w", encoding="utf-8") as f:
            f.write(website_data)

website_data

' Halloumi BurgerΜΠΙΦΤΕΚΙ ΜΟΣΧΑΡΙΣΙΟ , ΧΑΛΟΥΜΙ , ΡΟΚΑ , ΝΤΟΜΑΤΑ , ΜΑΓΙΟΝΕΖΑ €5.30 \n Number 1 BurgerΜΠΙΦΤΕΚΙ ΜΟΣΧΑΡΙΣΙΟ , ΔΙΠΛΟ CHEDDAR , ΑΥΓΟ ΤΗΓΑΝΙΤΟ , ΜΑΡΟΥΛΙ , ΝΤΟΜΑΤΑ , ΚΑΡΑΜΕΛΩΜΕΝΑ ΚΡΕΜΜΥΔΙΑ €5.50 \n Shrimp BurgerΠΑΝΑΡΙΣΜΕΝΗ ΓΑΡΙΔΑ , ΝΤΟΜΑΤΑ , ΚΡΕΜΜΥΔΙ , MΑΡΟΥΛΙ ,\xa0 ΑΒΟΚΑΝΤΟ , ΣΩΣ COCKTAIL €5.20 \n Φτιάξε το δικό σου burger! €0.80 \n HamburgerΜΠΙΦΤΕΚΙ, ΝΤΟΜΑΤΑ, ΚΕΤΣΑΠ, ΑΓΓΟΥΡΟΜΑΓΙΟΝΕΖΑ €3.60 \n Cheese burgerΜΠΙΦΤΕΚΙ, ΤΥΡΙ, ΝΤΟΜΑΤΑ, ΚΕΤΣΑΠ, ΑΓΓΟΥΡΟΜΑΓΙΟΝΕΖΑ €3.90 \n Super burgerΜΠΙΦΤΕΚΙ, ΝΤΟΜΑΤΑ, ΜΠΕΪΚΟΝ, ΠΑΡΜΕΖΑΝΑ, ΚΕΤΣΑΠ, ΑΓΓΟΥΡΟΜΑΓΙΟΝΕΖΑ €4.00 \n Mega burgerΔΙΠΛΟ ΜΠΙΦΤΕΚΙ, ΔΙΠΛΟ ΤΥΡΙ, ΜΠΕΪΚΟΝ, ΜΑΡΟΥΛΙ, ΚΡΕΜΜΥΔΙ, ΝΤΟΜΑΤΑ, ΚΕΤΣΑΠ, ΜΑΓΙΟΝΕΖΑ. ΣYNOΔEYETAI ME ΠΑΤΑΤΕΣ, ΣΩΣ €7.90 \n Chicken burgerΜΠΙΦΤΕΚΙ ΚΟΤΟΠΟΥΛΟ, ΤΥΡΙ, ΝΤΟΜΑΤΑ, ΜΑΡΟΥΛΙ, ΣΩΣ €4.00 \n Golden chickenΣΝΙΤΣΕΛ ΚΟΤΟΠΟΥΛΟ, ΠΑΡΜΕΖΑΝΑ, ΝΤΟΜΑΤΑ, ΜΑΡΟΥΛΙ, ΜΑΓΙΟΝΕΖΑ €4.00 \n VeggieΜΠΙΦΤΕΚΙ ΛΑΧΑΝΙΚΩΝ, ΑΓΓΟΥΡΙ, ΜΑΡΟΥΛΙ, ΝΤΟΜΑΤΑ, ΚΑΡΑΜΕΛΩΜΕΝΑ ΚΡΕΜΜΥΔΙΑ €3.90 \n Donuts τυρί-ζαμπόν €2.70 \n Sandwich με γύρο κοτόπουλο[ΓΥΡΟΣ ΚΟΤΟΠΟΥ

In [8]:
with open("final_GPT_output.txt", "w", encoding="utf-8") as f:
            f.write(final_data)
            
final_data

'halloumi burger (μπιφτέκι μοσχαρίσιο, χαλούμι, ρόκα, ντομάτα, μαγιονέζα) - €5.30\nnumber 1 burger (μπιφτέκι μοσχαρίσιο, διπλό cheddar, αυγό τηγανητό, μαρούλι, ντομάτα, καραμελωμένα κρεμμύδια) - €5.50\nshrimp burger (παναρισμένη γαρίδα, ντομάτα, κρεμμύδι, μαρούλι, αβοκάντο, σως cocktail) - €5.20\nφτιάξε το δικό σου burger! - €0.80\nhamburger (μπιφτέκι, ντομάτα, κέτσαπ, αγγουρομαγιονέζα) - €3.60\ncheese burger (μπιφτέκι, τυρί, ντομάτα, κέτσαπ, αγγουρομαγιονέζα) - €3.90\nsuper burger (μπιφτέκι, ντομάτα, μπέικον, παρμεζάνα, κέτσαπ, αγγουρομαγιονέζα) - €4.00\nmega burger (διπλό μπιφτέκι, διπλό τυρί, μπέικον, μαρούλι, κρεμμύδι, ντομάτα, κέτσαπ, μαγιονέζα. συνοδεύεται με πατάτες, σως) - €7.90\nchicken burger (μπιφτέκι κοτόπουλο, τυρί, ντομάτα, μαρούλι, σως) - €4.00\ngolden chicken (σνίτσελ κοτόπουλο, παρμεζάνα, ντομάτα, μαρούλι, μαγιονέζα) - €4.00\nveggie (μπιφτέκι λαχανικών, αγγούρι, μαρούλι, ντομάτα, καραμελωμένα κρεμμύδια) - €3.90\ndonuts τυρί-ζαμπόν - €2.70\nsandwich με γύρο κοτόπουλο - 

Total price for 50k tokens ~0.15$